In [1]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
%matplotlib inline

In [6]:
from pyhive import hive

# Set python variables from environment variables
username = os.environ['RENKU_USERNAME']
hive_host = os.environ['HIVE_SERVER2'].split(':')[0]
hive_port = os.environ['HIVE_SERVER2'].split(':')[1]

# create connection
conn = hive.connect(host=hive_host,
                    port=hive_port,
                    username=username) 
# create cursor
cur = conn.cursor()

In [14]:
query = """
    drop database if exists {0} cascade
""".format(username)
cur.execute(query)

query = """
    create database {0} location "/user/{0}/hive"
""".format(username)
cur.execute(query)

query = """
    use {0}
""".format(username)
cur.execute(query)

# Actual data

In [15]:
query = """
    drop table if exists {0}.sbb_orc
""".format(username)
cur.execute(query)

query = """
    create external table {0}.sbb_orc(
        BETRIEBSTAG string,
        FAHRT_BEZEICHNER string,
        BETREIBER_ID string,
        BETREIBER_ABK string,
        BETREIBER_NAME string,
        PRODUKT_ID string,
        LINIEN_ID string,
        LINIEN_TEXT string,
        UMLAUF_ID string,
        VERKEHRSMITTEL_TEXT string,
        ZUSATZFAHRT_TF string,
        FAELLT_AUS_TF string,
        BPUIC string,
        HALTESTELLEN_NAME string,
        ANKUNFTSZEIT string,
        AN_PROGNOSE string,
        AN_PROGNOSE_STATUS string,
        ABFAHRTSZEIT string,
        AB_PROGNOSE string,
        AB_PROGNOSE_STATUS string,
        DURCHFAHRT_TF string
    )
    stored as orc
    location '/data/sbb/part_orc/istdaten'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.sbb_orc limit 5
""".format(username)
pd.read_sql(query, conn)

,sbb_orc.betriebstag,sbb_orc.fahrt_bezeichner,sbb_orc.betreiber_id,sbb_orc.betreiber_abk,sbb_orc.betreiber_name,sbb_orc.produkt_id,sbb_orc.linien_id,sbb_orc.linien_text,sbb_orc.umlauf_id,sbb_orc.verkehrsmittel_text,sbb_orc.zusatzfahrt_tf,sbb_orc.faellt_aus_tf,sbb_orc.bpuic,sbb_orc.haltestellen_name,sbb_orc.ankunftszeit,sbb_orc.an_prognose,sbb_orc.an_prognose_status,sbb_orc.abfahrtszeit,sbb_orc.ab_prognose,sbb_orc.ab_prognose_status,sbb_orc.durchfahrt_tf
0,20.01.2018,80:06____:17010:000,80:06____,DB,DB Regio AG,Zug,17010,RE,,RE,false,false,8500090,Basel Bad Bf,,,PROGNOSE,20.01.2018 05:45,,UNBEKANNT,false
1,20.01.2018,80:06____:17012:000,80:06____,DB,DB Regio AG,Zug,17012,RE,,RE,false,false,8500090,Basel Bad Bf,,,PROGNOSE,20.01.2018 06:34,,UNBEKANNT,false
2,20.01.2018,80:06____:17014:000,80:06____,DB,DB Regio AG,Zug,17014,RE,,RE,false,false,8500090,Basel Bad Bf,,,PROGNOSE,20.01.2018 09:48,,UNBEKANNT,false
3,20.01.2018,80:06____:17015:000,80:06____,DB,DB Regio AG,Zug,17015,RE,,RE,false,false,8500090,Basel Bad Bf,20.01.2018 08:06,,UNBEKANNT,,,PROGNOSE,false
4,20.01.2018,80:06____:17016:000,80:06____,DB,DB Regio AG,Zug,17016,RE,,RE,false,false,8500090,Basel Bad Bf,,,PROGNOSE,20.01.2018 10:48,,UNBEKANNT,false


# Stations data

In [18]:
query = """
    drop table if exists {0}.stops
""".format(username)
cur.execute(query)

# STATIONID: identifier of the station/stop
# LONGITUDE: longitude (WGS84)
# LATITUDE: latitude (WGS84)
# HEIGHT: altitude (meters) of the stop
# REMARK: long name of the stop
    
query = """
    create external table {0}.stops(
        STOP_ID string,
        STOP_NAME string,
        STOP_LAT string,
        STOP_LON string
    )
    stored as orc
    location '/data/sbb/orc/allstops'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.stops limit 5
""".format(username)
pd.read_sql(query, conn)

,stops.stationid,stops.longitude,stops.latitude,stops.height,stops.remark
0,1100008,"Zell (Wiesental), Wilder Mann",47.7100842702352,7.85964788274668,
1,1100009,"Zell (Wiesental), Grönland",47.7131911044794,7.86290876722849,
2,1100010,Atzenbach,47.7146175266411,7.8723500608659,
3,1100011,"Mambach, Brücke",47.7282088873189,7.8774704579861,
4,1100012,"Mambach, Mühlschau",47.7340818684375,7.8813871126254,


# Timetable data

In [19]:
query = """
    drop table if exists {0}.stops
""".format(username)
cur.execute(query)

# TRIP_ID: identifier (FK) of the trip, unique for the day - e.g. 1.TA.1-100-j19-1.1.H
# ARRIVAL_TIME: scheduled (local) time of arrival at the stop (same as DEPARTURE_TIME if this is the start of the journey)
# DEPARTURE_TIME: scheduled (local) time of departure at the stop
# STOP_ID: stop (station) identifier (FK), from stops.txt
# STOP_SEQUENCE: sequence number of the stop on this trip id, starting at 1.
# PICKUP_TYPE:
# DROP_OFF_TYPE:
    
query = """
    create external table {0}.stop_times(
        TRIP_ID string,
        ARRIVAL_TIME string,
        DEPARTURE_TIME string,
        STOP_ID string,
        STOP_SEQUENCE string,
        PICKUP_TYPE string,
        DROP_OFF_TYPE string
    )
    stored as orc
    location '/data/sbb/part_orc/stop_times'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.stop_times limit 5
""".format(username)
pd.read_sql(query, conn)

,stop_times.trip_id,stop_times.arrival_time,stop_times.departure_time,stop_times.stop_id,stop_times.stop_sequence,stop_times.pickup_type,stop_times.drop_off_type
0,1.TA.1-88-j18-1.1.R,05:45:00,05:45:00,8572313,1,0,0
1,1.TA.1-88-j18-1.1.R,05:46:00,05:46:00,8572258,2,0,0
2,1.TA.1-88-j18-1.1.R,05:49:00,05:49:00,8580475,3,0,0
3,1.TA.1-88-j18-1.1.R,05:49:00,05:49:00,8572256,4,0,0
4,1.TA.1-88-j18-1.1.R,05:51:00,05:51:00,8582231,5,0,0


In [20]:
query = """
    drop table if exists {0}.trips
""".format(username)
cur.execute(query)

# ROUTE_ID: identifier (FK) for the route. A route is a sequence of stops. It is time independent.
# SERVICE_ID: identifier (FK) of a group of trips in the calendar, and for managing exceptions (e.g. holidays, etc).
# TRIP_ID: is one instance (PK) of a vehicle journey on a given route - the same route can have many trips at regular intervals; a trip may skip some of the route stops.
# TRIP_HEADSIGN: displayed to passengers, most of the time this is the (short) name of the last stop.
# TRIP_SHORT_NAME: internal identifier for the tripheadsign (note TRIPHEADSIGN and TRIPSHORTNAME are only unique for an agency)
# DIRECTION_ID: if the route is bidirectional, this field indicates the direction of the trip on the route.
    
query = """
    create external table {0}.trips(
        ROUTE_ID string,
        SERVICE_ID string,
        TRIP_ID string,
        TRIP_HEADSIGN string,
        TRIP_SHORT_NAME string,
        DIRECTION_ID string
    )
    stored as orc
    location '/data/sbb/part_orc/trips'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.trips limit 5
""".format(username)
pd.read_sql(query, conn)

,trips.route_id,trips.service_id,trips.trip_id,trips.trip_headsign,trips.trip_short_name,trips.direction_id
0,1-9-j18-1,TA+b0001,4.TA.1-9-j18-1.3.H,Lenzburg,21916,0
1,1-ZB1-j18-1,TA+b0001,1.TA.1-ZB1-j18-1.1.R,"Bad Zurzach, Bahnhof",101,1
2,1-ZB1-j18-1,TA+b0001,2.TA.1-ZB1-j18-1.1.R,"Bad Zurzach, Bahnhof",103,1
3,1-ZB1-j18-1,TA+b0001,3.TA.1-ZB1-j18-1.1.R,"Bad Zurzach, Bahnhof",105,1
4,1-ZB1-j18-1,TA+b0001,4.TA.1-ZB1-j18-1.1.R,"Bad Zurzach, Bahnhof",107,1
